# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.model import Model
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset - Heart Failure Clinical Records Dataset

### Overview
Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

### Objective
Train a model to predict whether the person with Cardiovascular disease will survive or not

In [ ]:
# Initialize Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'automl_heart_failure_prediction'
project_folder = './automl'

experiment=Experiment(ws, experiment_name)

In [2]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F6YCPV7ZF to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-137804
aml-quickstarts-137804
southcentralus
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


Name,Workspace,Report Page,Docs Page
automl_heart_failure_prediction,quick-starts-ws-137804,Link to Azure Machine Learning studio,Link to Documentation


## Create or Attach an AmlCompute cluster

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = compute_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

In [4]:
dataset = Dataset.get_by_name(ws, name='Heart_Failure_Clinical_Records_Dataset')

df = dataset.to_pandas_dataframe()

df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
project_folder = './automl'

In [20]:
if "automl" not in os.listdir():
    os.mkdir("./automl")

# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "n_cross_validations":5,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",
                             path = project_folder,
                             enable_early_stopping = True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [21]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_62097164-b25d-43d6-9bc1-d749f6dfa2f9',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T11:44:42.575945Z',
 'endTimeUtc': '2021-02-07T12:10:54.908614Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_heart_failure_prediction","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-137804","workspace_name":"quick-starts-ws-137804","region":"southcentralus","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":nu

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
automl_best_run, automl_fitted_model = remote_run.get_output()

In [ ]:
automl_best_run.get_metrics()

In [ ]:
print(automl_best_run)

In [ ]:
print(automl_fitted_model)

In [ ]:
# Retrieve and save your best automl model.

os.makedirs('outputs', exist_ok=True)

joblib.dump(value = automl_fitted_model, filename = 'output/best-automl.pkl')

# from azureml.automl.runtime.onnx_convert import OnnxConverter

# automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

# OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_automl_model.onnx')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model.register(workspace = ws,
                        model_path = 'output/best-automl.pkl',
                        model_name = 'automl-best-model')

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'automl-env.yml')

In [ ]:
env = Environment.from_conda_specification('automl-env', 'automl-env.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1,
                                                auth_enabled = True, 
                                                enable_app_insights = True,
                                                tags = {'name':'Heart failure prediction'},
                                                description='Heart failure prediction model')

service = model.deploy(workspace = ws, 
                       name = 'predict-heart-failure', 
                       models = [model], 
                       inference_config = inference_config,
                       deployment_config = aci_config)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.state)

In [ ]:
print("scoring URI: " + service.scoring_uri)

print("Swagger URI: " + service.swagger_uri)

print("Authetication Key: " + service.get_keys()[0])

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

scoringuri = service.scoring_uri
key = service.get_keys()[0]

data= { "data":
       [
           {
               'age': 74,
               'anaemia': 1,
               'creatinine_phosphokinase': 1618,
               'diabetes': 1,
               'ejection_fraction': 27,
               'high_blood_pressure': 1,
               'platelets': 275095,
               'serum_creatinine': 2.3,
               'serum_sodium': 133,
               'sex': 1,
               'smoking': 0,
               'time': 9
               
           },
           {
               'age': 52,
               'anaemia': 0,
               'creatinine_phosphokinase': 141,
               'diabetes': 0,
               'ejection_fraction': 19,
               'high_blood_pressure': 0,
               'platelets': 253000,
               'serum_creatinine': 0.9,
               'serum_sodium': 129,
               'sex': 0,
               'smoking': 1,
               'time': 10
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [1]:
service.delete()